# Emitter Spatial Clustering Analysis

## Imports

In [ ]:
import os
import sys
import platform
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from sklearn.mixture import GaussianMixture
from matplotlib.colors import Normalize, LogNorm, LinearSegmentedColormap

In [ ]:
# Reproducibility & performance control
os.environ["OMP_NUM_THREADS"] = "1"   # avoid oversubscription warnings
print("Python:", sys.version)
print("Platform:", platform.platform())

#### Functions

In [ ]:
def inlier_mask_mahalanobis(pts, labels, means, covs, nsig=3.0, eps=1e-9):
    """
    Returns a boolean mask marking points that are within nsig Mahalanobis
    stds of their assigned GMM component mean.
    """
    K = len(means)
    keep = np.zeros(len(pts), dtype=bool)
    thr2 = float(nsig) ** 2  # squared threshold, e.g. 9 for 3σ
    idx = (labels == 0)
    mu = means[0]
    cov_k = covs[0]
    if cov_k.ndim == 1:
        cov_k = np.diag(cov_k)
    # Regularize in case of near-singular covariance
    cov_k = cov_k + eps * np.eye(cov_k.shape[0])
    inv_cov = np.linalg.inv(cov_k)
    dx = pts[idx] - mu
    # Squared Mahalanobis distance for each point
    d2 = np.sum((dx @ inv_cov) * dx, axis=1)
    keep[idx] = d2 <= thr2
    return keep

In [ ]:
# Helper: draw an n-std ellipse from a 2x2 covariance
def draw_cov_ellipse(mean, cov, ax, n_std=2.0, edgecolor='k', facecolor='none', lw=2, zorder=3):
    # Handle both 'full' and 'diag' forms
    if cov.ndim == 1:
        cov = np.diag(cov)
    # Eigen-decomposition
    vals, vecs = np.linalg.eigh(cov)
    # Sort by descending eigenvalue
    order = vals.argsort()[::-1]
    vals, vecs = vals[order], vecs[:, order]
    # Width/height are 2*n_std*sqrt(eigenvalues)
    width, height = 2 * n_std * np.sqrt(vals)
    # Angle of the largest eigenvector
    angle = np.degrees(np.arctan2(vecs[1, 0], vecs[0, 0]))
    e = Ellipse(xy=mean, width=width, height=height, angle=angle,
                edgecolor=edgecolor, facecolor=facecolor, lw=lw, zorder=zorder)
    ax.add_patch(e)

In [ ]:
def get_best_gmm_components(n, x, y, min_cluster_size=1, sigma_rms_threshold=6.0):
    "The following function fits GMMs with different component counts and selects the best one based on BIC."
    pts = np.column_stack([x, y])
    range_n = range(2, n+1) if len(pts) >= n else range(2, len(pts))
    bics, aics, models, min_counts = [], [], [], []
    for n in range_n:
        gmm = GaussianMixture(
            n_components=n,
            covariance_type='full',   # 'full' is flexible; 'diag' if you want axis-aligned
            n_init=10,
            random_state=42,
            reg_covar=1e-6            # avoids singular covariances
        )
        gmm.fit(pts)
        models.append(gmm)
        bics.append(gmm.bic(pts))
        aics.append(gmm.aic(pts))
        counts = np.bincount(gmm.predict(pts), minlength=n)
        min_counts.append(counts.min())
    
    sigma_rms_models = []
    for gmm in models:
        covs_g = gmm.covariances_
        if covs_g.ndim == 3:           # full covariance (k, d, d)
            sigmas = np.sqrt(np.trace(covs_g, axis1=1, axis2=2))
        else:                          # diag form (k, d)
            sigmas = np.sqrt(np.sum(covs_g, axis=1))
        sigma_rms_models.append(sigmas.astype(float))   # ensure numeric dtype

    bics = np.array(bics)
    aics = np.array(aics)
    min_counts = np.array(min_counts)

    # Prefer the model with smallest BIC satisfying:
# 1. No singleton clusters (min_counts >= min_cluster_size)
# 2. At least one component with sigma_rms > sigma_rms_threshold
    order = np.argsort(bics)
    def passes_all(i):
        return (min_counts[i] >= min_cluster_size and
            np.any(sigma_rms_models[i] < sigma_rms_threshold))
    # First try full criteria
    filtered = [i for i in order if passes_all(i)]

    if filtered:
        best_idx = filtered[0]
    else:
        # Relax: only min cluster size criterion
        filtered_size = [i for i in order if min_counts[i] >= min_cluster_size]
        if filtered_size:
            best_idx = filtered_size[0]
        else:
            # Final fallback: absolute best BIC
            best_idx = order[0]
    best_n = int(list(range_n)[best_idx])
    best_gmm = models[best_idx]
    chosen_sigmas = sigma_rms_models[best_idx]
    print(
    f"Selected n={best_n} by BIC with criteria: "
    f"min_cluster_size>={min_cluster_size}, "
    f"σ_rms_threshold={sigma_rms_threshold}. "
    f"(min size={min_counts[best_idx]}, σ_rms={np.round(chosen_sigmas,3)})")

    # Plot BIC/AIC curves
    plt.figure(figsize=(8,4))
    plt.plot(list(range_n), bics, '-o', label='BIC')
    plt.plot(list(range_n), aics, '-o', label='AIC', alpha=0.7)
    plt.scatter([best_n], [bics[best_idx]], c='r', zorder=3, label=f'best n={best_n}')
    plt.xlabel('n_components')
    plt.ylabel('score (lower is better)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    return best_n, best_gmm

#### Inputs

In [ ]:
# Parameters (edit here)
CELL_ID = 5
INTENSITY_QUANTILE_RANGE = (0.02, 0.98)
MAHAL_NSIG = 3.0
BIC_MAX_COMPONENTS = 5
SIGMA_RMS_THRESHOLD = 6.0
MIN_CLUSTER_SIZE = 3          # reject GMM solutions with singletons
RANDOM_STATE = 42

In [ ]:
base_prefix = r"c:\Users\97254\Desktop\Resources\Technion\grad_school\shechtman_lap\projects\MS2\outputs\gRNA2\gRNA2_12.03.25-st-13-II"
emitter_path = fr"{base_prefix}\v2\emitter_analysis\cell_{CELL_ID}_data_global_peaks_final.csv"
df = pd.read_csv(emitter_path)
print("Rows:", len(df))
df.head()

In [ ]:
from IPython.display import display, Markdown

def stop_if(cond, msg="Stopping notebook."):
    if cond:
        display(Markdown(f"### ⛔ {msg}"))
        raise SystemExit  # halts execution; Run All won’t continue

# Example:
stop_if(len(df) < 2,
        f"Amount of emitters detect are ({len(df)})")

#### Emitter Clustering

In [ ]:
dist = df["dist_to_center"].to_numpy()
angle = df["angle_to_center"].to_numpy()
x = dist * np.cos(angle)
y = dist * np.sin(angle)
pts = np.column_stack([x, y])
intensity = df["ellipse_sum"].to_numpy()

In [ ]:
gmm1 = GaussianMixture(n_components=1, random_state=RANDOM_STATE)
labels = gmm1.fit_predict(pts)
probs = gmm1.predict_proba(pts).max(axis=1)  # confidence per point
means = gmm1.means_
covs = gmm1.covariances_

In [ ]:
sigma_rms = np.sqrt(covs[0][0][0] + covs[0][1][1])   
print(f"sigma_rms: {sigma_rms}")

In [ ]:
keep_mask = inlier_mask_mahalanobis(pts, labels, means, covs, nsig=3.0)

In [ ]:
colors_list = [
    "#0d0887", "#6a00a8", "#b12a90", "#e16462", "#fca636", "#f0f921"
]
my_cmap = LinearSegmentedColormap.from_list("my_cmap", colors_list, N=256)
vmin, vmax = np.quantile(intensity, INTENSITY_QUANTILE_RANGE)
norm = Normalize(vmin=vmin, vmax=vmax, clip=True)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,6))
ax.scatter(x[keep_mask], y[keep_mask], c=intensity[keep_mask], cmap=my_cmap, s=10, alpha=0.5, label='Emitters')
ax.scatter(x[~keep_mask], y[~keep_mask], c='red', s=10, alpha=1.0, label='Filtered Emitters')
ax.scatter(0,0, c='black', s=50, marker='x', label='Cell Center')
ax.grid(True)
ax.set_title(f'GMM Clustering of Emitters for Cell {CELL_ID},rms_sigma={sigma_rms:.2f}')
draw_cov_ellipse(means[0], covs[0], ax, n_std=1.0, edgecolor='blue', lw=1)
draw_cov_ellipse(means[0], covs[0], ax, n_std=2.0, edgecolor='blue', lw=1)
draw_cov_ellipse(means[0], covs[0], ax, n_std=3.0, edgecolor='blue', lw=1)
cbar = plt.colorbar(plt.cm.ScalarMappable(norm=norm, cmap=my_cmap), ax=ax)
cbar.set_label('Emitter Intensity')
plt.legend()
plt.show()

In [ ]:
if sigma_rms > SIGMA_RMS_THRESHOLD:
    n_components, best_gmm = get_best_gmm_components(BIC_MAX_COMPONENTS, x, y, 2)
    labels = best_gmm.predict(pts)
    probs_n = best_gmm.predict_proba(pts).max(axis=1)  # confidence per point
    means_n = best_gmm.means_
    covs_n = best_gmm.covariances_
    sigma_rms_n = [np.sqrt(covs_n[i][0, 0] + covs_n[i][1, 1]) for i in range(n_components)]
    fig,ax = plt.subplots(1,1,figsize=(12,6))
    # Means and covariance ellipses (1σ and 2σ)
    colors = ['blue', 'red', 'green', 'orange', 'purple']
    for i, (m, c) in enumerate(zip(means_n, covs_n)):
        ax.scatter(x, y, c=labels, cmap='coolwarm', s=8, edgecolors='none', zorder=1)
        ax.scatter(m[0], m[1], c=colors[i], s=120, marker='x', linewidths=2, zorder=4)
        draw_cov_ellipse(m, c, ax, n_std=1.0, edgecolor=colors[i], lw=1.5, zorder=3)
        draw_cov_ellipse(m, c, ax, n_std=1.5, edgecolor=colors[i], lw=1.2, zorder=3)
        draw_cov_ellipse(m, c, ax, n_std=2.0, edgecolor=colors[i], lw=1.0, zorder=3)
    ax.set_title(f'GMM Clustering sigma_rms={[f"{s:.2f}" for s in sigma_rms_n]} for Cell {CELL_ID} with n={n_components}')
    ax.scatter(0,0, c='black', s=100, marker='+', linewidths=2)
    ax.scatter(means_n[:,0], means_n[:,1], c = colors[:n_components], s=150, marker='x', linewidths=2)
    ax.grid(True)
    plt.tight_layout()

In [ ]:
labels